In [1]:
import zipfile
import os
import random
from PIL import Image
from PIL import ImageEnhance
import json
import pandas as pd
import numpy as np
from sklearn import metrics

src_path="F:/UIC/Year 3(Right)/Machine Learning/Project/Rumor_Dataset.zip"
target_path="F:/UIC/Year 3(Right)/Machine Learning/Project/Chinese_Rumor_Dataset-master"
if(not os.path.isdir(target_path)):
    z = zipfile.ZipFile(src_path, 'r')
    z.extractall(path=target_path)
    z.close()

In [2]:
rumor_class_dirs = os.listdir(target_path+"/CED_Dataset/rumor-repost/")
non_rumor_class_dirs = os.listdir(target_path+"/CED_Dataset/non-rumor-repost/")
original_microblog = target_path+"/CED_Dataset/original-microblog/"

data_list_path="F:/UIC/Year 3(Right)/Machine Learning/Project/"
all_data_path=data_list_path + "all_data.txt"

train_list_path = data_list_path+'train_list.txt'
eval_list_path = data_list_path+'eval_list.txt'

In [3]:
train_data_origin = open(train_list_path, encoding='utf-8')
train_data_mid = pd.DataFrame(train_data_origin)
#train_data_mid

In [4]:
#use max len:
final_list = []
lenlist = []

#find max length
for i in range(train_data_mid.shape[0]):
    te2 = train_data_mid[0][i]
    md1 = te2.replace("\t", ",").replace("\n", "")
    te3 = list(map(int, md1.split(",")))
    te4 = np.array(te3).T
    lenlist.append(len(te4))
max_len = max(lenlist) - 1

for i in range(train_data_mid.shape[0]):
    te2 = train_data_mid[0][i]
    md1 = te2.replace("\t", ",").replace("\n", "")
    te3 = list(map(int, md1.split(",")))
    te4 = np.array(te3).T
    
    #get x
    fin_list = te4[0:-1]
    if len(fin_list) <= max_len:
        for j in range(max_len - len(te4[0:-1])):
            fin_list = np.append(fin_list, 0)  # add 0 to list
            
    #get y
    fin_list = np.append(fin_list, te4[-1])
    
    
    final_list.append(fin_list)

train_data = pd.DataFrame(final_list)
train_X = train_data.iloc[:,0:-1]
train_y = train_data.iloc[:,-1]

In [5]:
#use max len:
eval_data_origin = open(eval_list_path, encoding='utf-8')
eval_data_mid = pd.DataFrame(eval_data_origin)
final_list = []

new_max_len = 226
for i in range(eval_data_mid.shape[0]):
    te2 = eval_data_mid[0][i]
    md1 = te2.replace("\t", ",").replace("\n", "")
    te3 = list(map(int, md1.split(",")))
    te4 = np.array(te3).T
    
    #get x
    fin_list = te4[0:-1]
    if len(fin_list) <= new_max_len:
        for j in range(new_max_len - len(te4[0:-1])):
            fin_list = np.append(fin_list, 0)  # add 0 to list
            
    #get y
    fin_list = np.append(fin_list, te4[-1])
    
    
    final_list.append(fin_list)

eval_data = pd.DataFrame(final_list)
eval_X = eval_data.iloc[:,0:-1]
eval_y = eval_data.iloc[:,-1]

In [6]:
#use SVM:
import seaborn as sb
from sklearn import svm
from scipy.io import loadmat

In [7]:
fin_svc = svm.SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
                 decision_function_shape='ovr', degree=3, gamma=100, kernel='rbf',
                 max_iter=-1, probability=True, random_state=None, shrinking=True,
                 tol=0.001, verbose=False)

In [8]:
fin_svc.fit(train_X, train_y)
fin_svc.score(eval_X, eval_y)

0.5801886792452831

In [9]:
resu_SVM = fin_svc.predict(eval_X)
def performance(y_ture,y_pred):
    print("Accuracy Score is:",metrics.accuracy_score(y_ture, y_pred))
    print("Precision Score is:", metrics.precision_score(y_ture, y_pred))
    print("Recall Score is:",metrics.recall_score(y_ture, y_pred))
    print("F1 Score is:",metrics.f1_score(y_ture, y_pred))
    print("\nConfusion_matrix is:\n",metrics.confusion_matrix(y_ture, y_pred))

print("SVM:")
performance(eval_y, resu_SVM)

SVM:
Accuracy Score is: 0.5801886792452831
Precision Score is: 0.5658536585365853
Recall Score is: 1.0
F1 Score is: 0.7227414330218067

Confusion_matrix is:
 [[ 14 178]
 [  0 232]]


In [10]:
#SVM: accuracy and precision
SVM_comb = pd.DataFrame([resu_SVM, eval_y]).T
TP_S = 0
FN_S = 0
TN_S = 0
FP_S = 0

for i in range(SVM_comb.shape[0]):
    if list(SVM_comb.iloc[i, :]) == [1, 1]:
        TP_S += 1
    if list(SVM_comb.iloc[i, :]) == [0, 0]:
        TN_S += 1
    if list(SVM_comb.iloc[i, :]) == [1, 0]:
        FP_S += 1
    if list(SVM_comb.iloc[i, :]) == [0, 1]:
        FN_S += 1

print(f"precision = {TP_S / (TP_S + FP_S)}")
print(f"recall = {TP_S / (TP_S + FN_S)}")

precision = 0.5658536585365853
recall = 1.0


In [11]:
#use Logistic Regression
import scipy.optimize as opt

def sigmoid(z):
    y = 1 / (1 + np.exp(-z))
    return y

def cost(theta, X, y):
    ''' cost function for you to minimize'''
    m = X.shape[0]
    totalSum = 0
    for i in range(m):
        xi = X[i]
        totalSum += -y[i] * np.log(1 / (1 + np.exp(-sum(xi * theta)))) - (1 - y[i]) * np.log(1 - (1 / (1 + np.exp(-sum(xi * theta)))))
    fin = 1 / m * float(totalSum)  # J(theta)
    return fin

def regularized_cost(theta, X, y, l=1):

    #     '''you don't penalize theta_0'''
    theta_j1_to_n = theta[1:]
    m = X.shape[0]
    sumThe = 0
    for i in range(len(theta_j1_to_n)):
        sumThe += theta_j1_to_n[i] * theta_j1_to_n[i]
    #print(cost(theta, X, y))
    regularized_term = l / (2 * m) * sumThe# your code here
    #print(regularized_term)
    
    return cost(theta, X, y) + regularized_term

def gradient(theta, X, y):
    return np.dot(np.transpose(X), (sigmoid(np.dot(X, theta)) - y)) / len(X)

def regularized_gradient(theta, X, y, reg_lambda=1):
#     '''still, leave theta_0 alone'''
    #print(X)
    res = (reg_lambda/len(X)) * theta  # λ / m * θj
    res[0] = 0  # theta0 = 0
    return gradient(theta, X, y) + res

theta = np.zeros(train_X.shape[1])
nx = np.array(train_X)
ny = np.array(train_y)
#result = opt.minimize(fun = cost, x0 = theta, args = (nx, ny), method = 'TNC', jac = gradient)
result = opt.minimize(fun=regularized_cost, x0=theta, args=(nx, ny), method='TNC', jac=regularized_gradient)


def predict(theta, X):
    probability = 1 / (1 + np.exp(np.dot(-X, theta)))  # h_theta(x) # your code goes here
    return np.array((probability >= 0.5).astype(int))

C:\Users\UIC\AppData\Local\Temp/ipykernel_3772/72375366.py:14: RuntimeWarning: divide by zero encountered in log
  totalSum += -y[i] * np.log(1 / (1 + np.exp(-sum(xi * theta)))) - (1 - y[i]) * np.log(1 - (1 / (1 + np.exp(-sum(xi * theta)))))
C:\Users\UIC\AppData\Local\Temp/ipykernel_3772/72375366.py:14: RuntimeWarning: invalid value encountered in multiply
  totalSum += -y[i] * np.log(1 / (1 + np.exp(-sum(xi * theta)))) - (1 - y[i]) * np.log(1 - (1 / (1 + np.exp(-sum(xi * theta)))))


In [12]:
final_theta = result.x
#print(final_theta)
y_pred = predict(final_theta, np.array(eval_X))
#print(y_pred)

In [13]:
corr_num = 0
for i in range(len(y_pred)):
    if y_pred[i] == eval_y[i]:
        corr_num += 1
print(corr_num / len(y_pred))

0.5542452830188679


In [14]:
print("Logistic Regression:")
performance(eval_y, y_pred)

Logistic Regression:
Accuracy Score is: 0.5542452830188679
Precision Score is: 0.5811320754716981
Recall Score is: 0.6637931034482759
F1 Score is: 0.619718309859155

Confusion_matrix is:
 [[ 81 111]
 [ 78 154]]


In [15]:
###LG: accuracy and precision
LG_comb = pd.DataFrame([y_pred, eval_y]).T
TP_L = 0
FN_L = 0
TN_L = 0
FP_L = 0

for i in range(LG_comb.shape[0]):
    if list(LG_comb.iloc[i, :]) == [1, 1]:
        TP_L += 1
    if list(LG_comb.iloc[i, :]) == [0, 0]:
        TN_L += 1
    if list(LG_comb.iloc[i, :]) == [1, 0]:
        FP_L += 1
    if list(LG_comb.iloc[i, :]) == [0, 1]:
        FN_L += 1

print(f"precision = {TP_L / (TP_L + FP_L)}")
print(f"recall = {TP_L / (TP_L + FN_L)}")
#TP_L + FN_L + TN_L + FP_L == LG_comb.shape[0]

precision = 0.5811320754716981
recall = 0.6637931034482759


In [16]:
#EL
pred_list = []



for num in range(101):
    print(num)
    #bootstrap
    ind = np.random.choice(train_X.shape[0], size=train_X.shape[0], replace=True)
    #print(ind)
    new_train_X = train_X.iloc[ind]
    new_train_y = train_y.iloc[ind]
    
    #print(new_train_X)
    
    #train
    fin_svc = svm.SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
                 decision_function_shape='ovr', degree=3, gamma=100, kernel='rbf',
                 max_iter=-1, probability=True, random_state=None, shrinking=True,
                 tol=0.001, verbose=False)
    fin_svc.fit(new_train_X, new_train_y)
    
    pred_list.append(np.array(fin_svc.predict(eval_X)).T)
    #print(np.array(fin_svc.predict(eval_X)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [17]:
ten = 0
"ten" + "i"

'teni'

In [18]:
pred_resu = pd.DataFrame(pred_list).T
final_pred = []
for i in range(pred_resu.shape[0]):
    if sum(pred_resu.iloc[i]) > 50:
        final_pred.append(1)
    else:
        final_pred.append(0)

sum(final_pred == eval_y) / pred_resu.shape[0]

0.5801886792452831

In [19]:
print("EL:")
performance(eval_y, final_pred)

EL:
Accuracy Score is: 0.5801886792452831
Precision Score is: 0.5658536585365853
Recall Score is: 1.0
F1 Score is: 0.7227414330218067

Confusion_matrix is:
 [[ 14 178]
 [  0 232]]


In [37]:
###EL: accuracy and precision
EL_comb = pd.DataFrame([final_pred, eval_y]).T
TP_E = 0
FN_E = 0
TN_E = 0
FP_E = 0

for i in range(EL_comb.shape[0]):
    if list(EL_comb.iloc[i, :]) == [1, 1]:
        TP_E += 1
    if list(EL_comb.iloc[i, :]) == [0, 0]:
        TN_E += 1
    if list(EL_comb.iloc[i, :]) == [1, 0]:
        FP_E += 1
    if list(EL_comb.iloc[i, :]) == [0, 1]:
        FN_E += 1
FN_E / EL_comb.shape[0]
#print(f"precision = {TP_E / (TP_E+ FP_E)}")
#print(f"recall = {TP_E / (TP_E + FN_E)}")

0.5471698113207547

In [36]:
#EL
pred_list = []

import scipy.optimize as opt

def sigmoid(z):
    y = 1 / (1 + np.exp(-z))
    return y

def cost(theta, X, y):
    ''' cost function for you to minimize'''
    m = X.shape[0]
    totalSum = 0
    for i in range(m):
        xi = X[i]
        totalSum += -y[i] * np.log(1 / (1 + np.exp(-sum(xi * theta)))) - (1 - y[i]) * np.log(1 - (1 / (1 + np.exp(-sum(xi * theta)))))
    fin = 1 / m * float(totalSum)  # J(theta)
    return fin

def regularized_cost(theta, X, y, l=1):

    #     '''you don't penalize theta_0'''
    theta_j1_to_n = theta[1:]
    m = X.shape[0]
    sumThe = 0
    for i in range(len(theta_j1_to_n)):
        sumThe += theta_j1_to_n[i] * theta_j1_to_n[i]
    #print(cost(theta, X, y))
    regularized_term = l / (2 * m) * sumThe# your code here
    #print(regularized_term)
    
    return cost(theta, X, y) + regularized_term

def gradient(theta, X, y):
    return np.dot(np.transpose(X), (sigmoid(np.dot(X, theta)) - y)) / len(X)

def regularized_gradient(theta, X, y, reg_lambda=1):
#     '''still, leave theta_0 alone'''
    #print(X)
    res = (reg_lambda/len(X)) * theta  # λ / m * θj
    res[0] = 0  # theta0 = 0
    return gradient(theta, X, y) + res

def predict(theta, X):
    probability = 1 / (1 + np.exp(np.dot(-X, theta)))  # h_theta(x) # your code goes here
    return np.array((probability >= 0.5).astype(int))

for num in range(101):
    print(num)
    #bootstrap
    ind = np.random.choice(train_X.shape[0], size=train_X.shape[0], replace=True)
    #print(ind)
    new_train_X = train_X.iloc[ind]
    new_train_y = train_y.iloc[ind]
    
    #print(new_train_X)
    
    #train
    theta = np.zeros(train_X.shape[1])
    nx = np.array(new_train_X)
    ny = np.array(new_train_y)
    #result = opt.minimize(fun = cost, x0 = theta, args = (nx, ny), method = 'TNC', jac = gradient)
    result = opt.minimize(fun=regularized_cost, x0=theta, args=(nx, ny), method='TNC', jac=regularized_gradient)
    
    final_theta = result.x
    #print(final_theta)
    #y_pred = predict(final_theta, np.array(eval_X))
    #print(y_pred)
    
    pred_list.append(np.array(predict(final_theta, np.array(eval_X))).T)
    #print(np.array(fin_svc.predict(eval_X)))

0


C:\Users\UIC\AppData\Local\Temp/ipykernel_12948/3105511271.py:16: RuntimeWarning: divide by zero encountered in log
  totalSum += -y[i] * np.log(1 / (1 + np.exp(-sum(xi * theta)))) - (1 - y[i]) * np.log(1 - (1 / (1 + np.exp(-sum(xi * theta)))))
C:\Users\UIC\AppData\Local\Temp/ipykernel_12948/3105511271.py:16: RuntimeWarning: invalid value encountered in multiply
  totalSum += -y[i] * np.log(1 / (1 + np.exp(-sum(xi * theta)))) - (1 - y[i]) * np.log(1 - (1 / (1 + np.exp(-sum(xi * theta)))))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [38]:
pred_resu_EL = pd.DataFrame(pred_list).T
final_pred_EL = []
for i in range(pred_resu_EL.shape[0]):
    if sum(pred_resu_EL.iloc[i]) > 50:
        final_pred_EL.append(1)
    else:
        final_pred_EL.append(0)

#sum(final_pred_EL == eval_y) / final_pred_EL.shape[0]

print("EL:")
performance(eval_y, final_pred_EL)

EL:
Accuracy Score is: 0.5589622641509434
Precision Score is: 0.5836431226765799
Recall Score is: 0.6767241379310345
F1 Score is: 0.6267465069860279

Confusion_matrix is:
 [[ 80 112]
 [ 75 157]]
